In [1]:
from __future__ import annotations

import datetime
import os
import pickle
import signal
import logging
import traceback
import random
from io import StringIO
from timeit import default_timer as timer
from datetime import date
from typing import Any
import numpy as np
from bqskit import compile
from bqskit.compiler.task import CompilationTask
from bqskit.ir import Circuit
from bqskit.qis import UnitaryMatrix, CouplingGraph, PermutationMatrix
from bqskit.compiler import MachineModel
from bqskit.compiler import BasePass, Compiler
from bqskit.passes import *
from bqskit.ext import qiskit_to_bqskit, bqskit_to_qiskit, bqskit_to_pytket, pytket_to_bqskit
import qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap

ImportError: 

Unable to import bqskit.ext package.
Ensure that bqskit was installed with extension support.
To install bqskit with extension support try:
	pip install bqskit[ext]


In [ ]:
def lin_model(qubits: int) -> list:
    num_edges = qubits - 1
    edges = set()

    for i in range(num_edges):
        edges.add((i, i+1))

    return list(edges)

model = lin_model(3)
cp_map = CouplingMap(model)

In [ ]:
model

# small circuit synthesis example, less than 5 qubits

In [26]:
qc = QuantumCircuit(3)
qc.cswap(0,1,2)
#decompose to cx
qc = transpile(qc, optimization_level = 3, basis_gates = ['u3', 'cx'])

In [27]:
qc.draw()

global phase: 1.5701
                                                                           »
q_0: ────────────────────────────────────────────────────■─────────────────»
      ┌──────────────────┐       ┌──────────────────┐    │                 »
q_1: ─┤ U3(π/2,π/2,-π/2) ├──■────┤ U3(π/2,-π/2,π/2) ├────┼─────────────────»
     ┌┴──────────────────┤┌─┴─┐┌─┴──────────────────┴─┐┌─┴─┐┌─────────────┐»
q_2: ┤ U3(π/2,0,-1.5695) ├┤ X ├┤ U3(3.1403,-3π/4,π/2) ├┤ X ├┤ U3(0,0,π/4) ├»
     └───────────────────┘└───┘└──────────────────────┘└───┘└─────────────┘»
«                                                  ┌─────────────┐           
«q_0: ───────────────────────■───────────■─────────┤ U3(0,0,π/4) ├───■───────
«          ┌─────────────┐   │         ┌─┴─┐       ├─────────────┴┐┌─┴─┐┌───┐
«q_1: ──■──┤ U3(0,0,π/4) ├───┼─────────┤ X ├───────┤ U3(0,0,-π/4) ├┤ X ├┤ X ├
«     ┌─┴─┐├─────────────┴┐┌─┴─┐┌──────┴───┴──────┐└──────────────┘└───┘└─┬─┘
«q_2: ┤ X ├┤ U3(0,0,-π/4) ├┤ X ├┤ U3(π/2,0,-3π/4) ├───────────────────────■──
«     └───┘└──────────────┘└───┘└─────────────────┘

qiskit transpile result:

In [28]:
trans_qc = transpile(qc, coupling_map=cp_map, optimization_level = 3, basis_gates = ['u3', 'cx'])

In [29]:
trans_qc.draw()

global phase: 0.74445
             ┌───────────────────────┐         »
q_0 -> 0 ────┤ U3(π,2.4111,-0.73047) ├─────────»
             ├───────────────────────┤         »
q_2 -> 1 ────┤ U3(π,2.8126,-0.32895) ├──────■──»
         ┌───┴───────────────────────┴───┐┌─┴─┐»
q_1 -> 2 ┤ U3(0.0013233,-π/2,1.1271e-12) ├┤ X ├»
         └───────────────────────────────┘└───┘»
«                                                  ┌───────────────────────┐   »
«q_0 -> 0 ────────────────────────────────────■────┤ U3(π,2.4669,-0.67474) ├───»
«          ┌──────────────────────────────┐ ┌─┴─┐┌─┴───────────────────────┴──┐»
«q_2 -> 1 ─┤ U3(0.79558,-2.9998,-0.20118) ├─┤ X ├┤ U3(0.79558,1.3696,-2.9998) ├»
«         ┌┴──────────────────────────────┴┐└───┘└────────────────────────────┘»
«q_1 -> 2 ┤ U3(1.5708,0.0013226,-0.033464) ├───────────────────────────────────»
«         └────────────────────────────────┘                                   »
«                                               ┌──────────────────────┐     »
«q_0 -> 0 ───────────────────────────────────■──┤ U3(0,-2.7619,1.1911) ├─────»
«                   ┌────────────────┐     ┌─┴─┐├──────────────────────┤     »
«q_2 -> 1 ──■───────┤ U3(π/2,-π/4,0) ├─────┤ X ├┤ U3(π,1.3328,-2.5942) ├──■──»
«         ┌─┴─┐┌────┴────────────────┴────┐└───┘└──────────────────────┘┌─┴─┐»
«q_1 -> 2 ┤ X ├┤ U3(1.5699,2.3897,3.1406) ├─────────────────────────────┤ X ├»
«         └───┘└──────────────────────────┘                             └───┘»
«                                                                        »
«q_0 -> 0 ───────────────────────────────────────────────────────────────»
«              ┌──────────────────┐            ┌────────────────┐        »
«q_2 -> 1 ─────┤ U3(π/2,-π/2,π/2) ├──────■─────┤ U3(π/2,0,-π/2) ├─────■──»
«         ┌────┴──────────────────┴───┐┌─┴─┐┌──┴────────────────┴──┐┌─┴─┐»
«q_1 -> 2 ┤ U3(0.0018714,-3π/4,-3π/4) ├┤ X ├┤ U3(1.5695,1.5695,-π) ├┤ X ├»
«         └───────────────────────────┘└───┘└──────────────────────┘└───┘»
«                                       ┌──────────────────────┐      »
«q_0 -> 0 ─────────────────────────■────┤ U3(π,0.84253,2.4133) ├───■──»
«          ┌────────────────────┐┌─┴─┐┌─┴──────────────────────┴┐┌─┴─┐»
«q_2 -> 1 ─┤ U3(3.1403,π/2,π/2) ├┤ X ├┤ U3(1.5712,1π/4,-1.5699) ├┤ X ├»
«         ┌┴────────────────────┤└───┘└─────────────────────────┘└───┘»
«q_1 -> 2 ┤ U3(1.6576,-π/2,π/2) ├─────────────────────────────────────»
«         └─────────────────────┘                                     »
«         ┌─────────────────────┐                          
«q_0 -> 0 ┤ U3(π,2.8449,2.0595) ├──────────────────────────
«         └┬───────────────────┬┘       ┌───────────────┐  
«q_2 -> 1 ─┤ U3(1.5695,-π,π/2) ├───■────┤ U3(π/2,-π,-π) ├──
«          └───────────────────┘ ┌─┴─┐┌─┴───────────────┴─┐
«q_1 -> 2 ───────────────────────┤ X ├┤ U3(π/2,-1.659,-π) ├
«                                └───┘└───────────────────┘

In [30]:
#qiskit generates circuits with 10 cnots
trans_qc.count_ops()

OrderedDict([('u3', 23), ('cx', 10)])

Qsearch synthesis:

In [31]:
bqskit_circ = qiskit_to_bqskit(qc)

In [32]:
inst_ops = instantiate_options={'multistarts':2, 'ftol':5e-16, 'gtol':1e-15}

In [45]:
task = CompilationTask(bqskit_circ.copy(), [
        SetModelPass(MachineModel(3, CouplingGraph.linear(3))),
        QSearchSynthesisPass(instantiate_options=inst_ops),
    ])

In [46]:
with Compiler() as compiler:
    opt_circuit = compiler.compile(task)

In [47]:
opt_circuit

Circuit(3)
	[U3Gate([3.141592653589793, 2.846874424730188, 4.665740060156603])@(0,), U3Gate([2.2045962222229796, 1.3710106003609661, 1.939288648746714])@(1,), U3Gate([1.333665027970923, 4.547118562580896, 2.097795735648832])@(2,)]
	[CNOTGate([])@(0, 1), CNOTGate([])@(0, 1), None]
	[U3Gate([3.141592653589793, 4.024420327349523, 1.6758567339076238])@(0,), U3Gate([0.7630908094628832, 5.770832105167222, 1.5707963322987015])@(1,), None]
	[None, CNOTGate([])@(1, 2), CNOTGate([])@(1, 2)]
	[None, U3Gate([1.570796326005353, 1.6670971334400027, 2.0831495229825983])@(1,), U3Gate([3.2769167204048366, 0.888460583349005, 3.252791166554247])@(2,)]
	[None, CNOTGate([])@(1, 2), CNOTGate([])@(1, 2)]
	[None, U3Gate([5.129129443153423, 4.712388979105864, 2.259893684083176])@(1,), U3Gate([0.22022640586175757, 6.96615430752403, 0.40310120781900954])@(2,)]
	[CNOTGate([])@(0, 1), CNOTGate([])@(0, 1), None]
	[U3Gate([3.141592653589793, 4.129385103589741, 4.580115806332114])@(0,), U3Gate([4.194505055313369, 1.1

In [48]:
opt_circuit.num_operations

27

In [49]:
#8CNOT circuit with qsearch synthesis
for gate in opt_circuit.gate_set:
    print(opt_circuit.count(gate), gate.name)

19 U3Gate
8 CNOTGate


In [50]:
distance = opt_circuit.get_unitary().get_distance_from(bqskit_circ.get_unitary())
distance

0.0

# Scalable synthesis requires first partition the circuit into blocks, then synthesis them individually, the flow is integrated in compile()

In [51]:
#you can generate the circuit in qiskit or load the qasm
adder_circuit = Circuit.from_file("adder9.qasm")

In [66]:
linear_model = MachineModel(16, coupling_graph=CouplingGraph.linear(16))
out_circuit = compile(adder_circuit, model=linear_model, optimization_level = 2)
print("Gate Counts:", out_circuit.gate_counts)
print("Connectivity:", out_circuit.coupling_graph)

Gate Counts: {U3Gate: 140, CNOTGate: 93}
Connectivity: CouplingGraph({(1, 2), (3, 4), (2, 3), (6, 7), (4, 5), (8, 9), (5, 6), (7, 8)})


In [58]:
#qiskit:
qiskit_adder = QuantumCircuit.from_qasm_file("adder9.qasm")

In [62]:
qiskit_adder.count_ops()

OrderedDict([('cx', 98), ('u1', 52), ('u2', 8), ('u3', 4)])

In [60]:
cp_map = CouplingMap(CouplingGraph.linear(16))

In [64]:
trans_adder = transpile(qiskit_adder, coupling_map = cp_map, optimization_level = 3, basis_gates = ['u3', 'cx'])

In [65]:
trans_adder.count_ops()

OrderedDict([('u3', 199), ('cx', 108)])